## Import Libraries and Declare API Key

In [1]:
import requests
import pandas as pd
import numpy as np
import signal
import time

In [2]:
API_KEY = {'X-API-key': 'RPCJTKLW'}

## Current Heat Trading Algorithm

### Basic Buy and Sell Functions

In [3]:
shutdown = False

class ApiException(Exception):
    pass

def signal_handler(signum, frame):
    global shutdown
    signal.signal(signal.SIGINT, signal.SIG_DFL)
    shutdown = True

def get_tick(session):
    resp = session.get('http://localhost:9999/v1/case')
    if resp.ok:
        case = resp.json()
        return case['tick']
    raise ApiException('Authorization error. Please check API key.')

def ticker_bid_ask(session, ticker):
    payload = {'ticker': ticker}
    resp = session.get('http://localhost:9999/v1/securities/book', params = payload)
    if resp.ok:
        book = resp.json()
        return book['bids'][0]['price'], book['asks'][0]['price']
    raise ApiException('Authorization error. Please check API key.')

def buy(session, ticker, type, q, price = None):
    if type == 'MARKET':
        resp = session.post('http://localhost:9999/v1/orders', params = {'ticker': ticker, 'type': 'MARKET', 'quantity': q, 'action': 'BUY'})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')
    else:
        resp = session.post('http://localhost:9999/v1/orders',
                            params={'ticker': ticker, 'type': 'LIMIT', 'quantity': q, 'action': 'BUY', 'price': price})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')

def sell(session, ticker, type, q, price = None):
    if type == 'MARKET':
        resp = session.post('http://localhost:9999/v1/orders', params = {'ticker': ticker, 'type': 'MARKET', 'quantity': q, 'action': 'SELL'})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')
    else:
        resp = session.post('http://localhost:9999/v1/orders',
                            params={'ticker': ticker, 'type': 'LIMIT', 'quantity': q, 'action': 'SELL', 'price': price})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')

def cancel(session, order_id):
    order_id = order_id # assuming the order to cancel has ID 100
    resp = session.delete('http://localhost:9999/v1/orders/{}'.format(order_id))
    if resp.ok:
        status = resp.json()
        success = status['success']
        print('The order was successfully cancelled?', success)
        
def tender_list(session):
    resp = session.get('http://localhost:9999/v1/tenders')
    if resp.ok:
        lst = resp.json()
        if lst:
            return lst[0]
        else:
            return None
        
def acc_tender(session, tender, price = None):
    if tender['is_fixed_bid'] is True:
        resp = session.post('http://localhost:9999/v1/tenders/{}'.format(tender['tender_id']), 
                           params = {'id': tender['tender_id']})
        if resp.ok:
            print('tender accepted!')
    else:
        resp = session.post('http://localhost:9999/v1/tenders/{}'.format(tender['tender_id']), 
                           params = {'price': price, 'id': tender['tender_id']})
        if resp.ok:
            print('tender accepted!')
      
    
def dec_tender(session, tender):
    resp = session.delete('http://localhost:9999/v1/tenders/{}'.format(tender['tender_id']), 
                           params = {'id': tender['tender_id']})
    if resp.ok:
        print('tender declined!')
        
def current_price(session, ticker):
    resp = s.get('http://localhost:9999/v1/securities', params = {'ticker': ticker})
    json = resp.json()
    return json[0]['last']

In [4]:
def determine_buy(session, tender, tender_vol, pbull, pbear, prutc, pusd):
    conv = 1 / pusd
    eq1 = tender_vol * pbull + tender_vol * pbear + 1500 * conv * (tender_vol/10000)
    eq2 = tender_vol * prutc * conv
    if eq1 > eq2:
        print('acc buy')
        acc_tender(session, tender)
    else:
        print('dec buy')
        dec_tender(session, tender)
    
def determine_sell(session, tender, tender_vol, pbull, pbear, prutc, pusd):
    conv = 1 / pusd
    eq1 = tender_vol * pbull + tender_vol * pbear + 1500 * conv * (tender_vol/10000)
    eq2 = tender_vol * prutc * conv
    if eq1 < eq2:
        print('acc sell')
        acc_tender(session, tender)
    else:
        print('dec sell')
        dec_tender(session, tender)

## Algorithm Execution Section

In [5]:
signal.signal(signal.SIGINT, signal_handler)

while True:
    with requests.Session() as s:
        s.headers.update(API_KEY)
        tick = get_tick(s) 
        print(tick)
        print(tender_list(s))
        if tender_list(s) and (tick == tender_list(s)['expires'] - 1 or tick == tender_list(s)['expires']):
            if tender_list(s)['action'] == 'BUY':
                print('buy?')
                determine_buy(s, tender_list(s), tender_list(s)['quantity'], current_price(s, 'BULL'),
                              current_price(s, 'BEAR'), current_price(s, 'RUTC'),  current_price(s, 'USD'))
            else:
                print('sell?')
                determine_sell(s, tender_list(s), tender_list(s)['quantity'], current_price(s, 'BULL'),
                              current_price(s, 'BEAR'), current_price(s, 'RUTC'), current_price(s, 'USD'))
    time.sleep(1)
    

0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
0
None
1
None
2
None
3
None
4
None
5
None
6
None
7
None
8
None
9
None
10
None
11
None
12
None
13
None
14
None
15
None
16
{'tender_id': 3, 'period': 1, 'tick': 15, 'expires': 21, 'caption': 'An institution would like to BUY 85000 shares of RUTC from you at a price of 25.24.  Would you like to SELL the shares to them?', 'ticker': 'RUTC', 'quantity': 85000.0, 'action': 'SELL', 'is_fixed_bid': True, 'price': 25.24}
17
{'tender_id': 3, 'period': 1, 'tick': 15, 'expires': 21, 'caption': 'An institution would like to BUY 85000 shares of RUTC from you at a price of 25.24.  Would you like to SELL the shares to

144
{'tender_id': 2003, 'period': 1, 'tick': 139, 'expires': 145, 'caption': 'An institution would like to SELL 95000 shares of RUTC to you at a price of 25.79.  Would you like to BUY the shares from them?', 'ticker': 'RUTC', 'quantity': 95000.0, 'action': 'BUY', 'is_fixed_bid': True, 'price': 25.79}
buy?
dec buy
tender declined!
145
None
146
None
147
None
148
None
149
None
150
None
151
None
152
None
153
None
154
None
155
None
156
None
157
None
158
None
159
None
161
None
162
None
163
None
164
None
165
None
166
None
167
None
168
None
169
None
170
None
171
None
172
None
173
None
174
None
175
None
176
None
177
{'tender_id': 2503, 'period': 1, 'tick': 176, 'expires': 182, 'caption': 'An institution would like to SELL 84000 shares of RUTC to you at a price of 25.78.  Would you like to BUY the shares from them?', 'ticker': 'RUTC', 'quantity': 84000.0, 'action': 'BUY', 'is_fixed_bid': True, 'price': 25.78}
178
{'tender_id': 2503, 'period': 1, 'tick': 176, 'expires': 182, 'caption': 'An instit

tender accepted!
31
None
32
None
33
None
34
None
35
None
36
None
37
None
38
None
39
None
40
None
41
None
42
None
43
None
44
None
45
None
46
None
47
None
48
None
49
None
50
None
51
None
52
None
53
None
54
None
55
None
56
{'tender_id': 503, 'period': 1, 'tick': 55, 'expires': 61, 'caption': 'An institution would like to BUY 77000 shares of RUTC from you at a price of 25.45.  Would you like to SELL the shares to them?', 'ticker': 'RUTC', 'quantity': 77000.0, 'action': 'SELL', 'is_fixed_bid': True, 'price': 25.45}
57
{'tender_id': 503, 'period': 1, 'tick': 55, 'expires': 61, 'caption': 'An institution would like to BUY 77000 shares of RUTC from you at a price of 25.45.  Would you like to SELL the shares to them?', 'ticker': 'RUTC', 'quantity': 77000.0, 'action': 'SELL', 'is_fixed_bid': True, 'price': 25.45}
58
{'tender_id': 503, 'period': 1, 'tick': 55, 'expires': 61, 'caption': 'An institution would like to BUY 77000 shares of RUTC from you at a price of 25.45.  Would you like to SELL the

175
{'tender_id': 2503, 'period': 1, 'tick': 170, 'expires': 176, 'caption': 'An institution would like to SELL 92000 shares of RUTC to you at a price of 24.19.  Would you like to BUY the shares from them?', 'ticker': 'RUTC', 'quantity': 92000.0, 'action': 'BUY', 'is_fixed_bid': True, 'price': 24.19}
buy?
acc buy
tender accepted!
176
None
177
None
178
None
179
None
180
None
181
None
182
None
183
None
184
None
185
None
186
None
187
None
188
None
189
None
190
None
191
None
193
None
194
None
195
None
196
None
197
None
198
None
199
None
200
None
201
None
202
None
203
None
204
None
205
None
207
{'tender_id': 3003, 'period': 1, 'tick': 206, 'expires': 212, 'caption': 'An institution would like to SELL 100000 shares of RUTC to you at a price of 24.68.  Would you like to BUY the shares from them?', 'ticker': 'RUTC', 'quantity': 100000.0, 'action': 'BUY', 'is_fixed_bid': True, 'price': 24.68}
208
{'tender_id': 3003, 'period': 1, 'tick': 206, 'expires': 212, 'caption': 'An institution would like

85
{'tender_id': 1003, 'period': 1, 'tick': 83, 'expires': 89, 'caption': 'An institution would like to SELL 87000 shares of RUTC to you at a price of 25.34.  Would you like to BUY the shares from them?', 'ticker': 'RUTC', 'quantity': 87000.0, 'action': 'BUY', 'is_fixed_bid': True, 'price': 25.34}
86
{'tender_id': 1003, 'period': 1, 'tick': 83, 'expires': 89, 'caption': 'An institution would like to SELL 87000 shares of RUTC to you at a price of 25.34.  Would you like to BUY the shares from them?', 'ticker': 'RUTC', 'quantity': 87000.0, 'action': 'BUY', 'is_fixed_bid': True, 'price': 25.34}
87
{'tender_id': 1003, 'period': 1, 'tick': 83, 'expires': 89, 'caption': 'An institution would like to SELL 87000 shares of RUTC to you at a price of 25.34.  Would you like to BUY the shares from them?', 'ticker': 'RUTC', 'quantity': 87000.0, 'action': 'BUY', 'is_fixed_bid': True, 'price': 25.34}
88
{'tender_id': 1003, 'period': 1, 'tick': 83, 'expires': 89, 'caption': 'An institution would like to

tender accepted!
205
None
206
None
207
None
208
None
209
None
210
None
211
None
212
None
213
None
214
None
215
None
216
None
217
None
218
None
219
None
220
None
221
None
222
None
223
None
224
None
225
None
226
None
227
None
228
None
229
None
230
None
231
None
232
{'tender_id': 3503, 'period': 1, 'tick': 231, 'expires': 237, 'caption': 'An institution would like to BUY 76000 shares of RUTC from you at a price of 25.72.  Would you like to SELL the shares to them?', 'ticker': 'RUTC', 'quantity': 76000.0, 'action': 'SELL', 'is_fixed_bid': True, 'price': 25.72}
233
{'tender_id': 3503, 'period': 1, 'tick': 231, 'expires': 237, 'caption': 'An institution would like to BUY 76000 shares of RUTC from you at a price of 25.72.  Would you like to SELL the shares to them?', 'ticker': 'RUTC', 'quantity': 76000.0, 'action': 'SELL', 'is_fixed_bid': True, 'price': 25.72}
234
{'tender_id': 3503, 'period': 1, 'tick': 231, 'expires': 237, 'caption': 'An institution would like to BUY 76000 shares of RUTC fr

ApiException: Authorization error. Please check API key.